<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 5:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline

In [2]:
import scrapy
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [64]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
response = requests.get(url)
HTML = response.text
HTML[0:500]

u'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of United States cities by population - Wikipedia, the free encyclopedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_'

In [88]:
top_50 = Selector(text=HTML).xpath('//div[@class="floatnone"]/a/@title').extract()
print top_50[:3]

def parser(uni):
    # RAW: u'1.  New York, New York'
    l = str(uni).split('.  ')
    return l[1].split(', ')

top_50 = map(parser, top_50)

[u'1.  New York, New York', u'2.  Los Angeles, California', u'3.  Chicago, Illinois']


In [87]:
pd.DataFrame(top_50, columns=['City','State'])

,City,State
0,New York,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania
5,Phoenix,Arizona
6,San Antonio,Texas
7,San Diego,California
8,Dallas,Texas
9,San Jose,California


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

In [90]:
# ONLY RETAIN PROPERLY FORMED CITIES WITH FILTERING FUNCTION
# I HAVE DONE

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [98]:
# https://www.craigslist.org/about/sites#US
# choose New York: https://newyork.craigslist.org/
# https://newyork.craigslist.org/search/sss?sort=rel&query=rv

def get_prices_from_craigslist(keyword, region):
    url = 'https://' + region +'.craigslist.org/search/sss?sort=rel&query=' + keyword
    print url
    
    response = requests.get(url)
    html = response.text
    
    px = Selector(text=html).xpath('//span[@class="price"]/text()').extract()
    return map(lambda p: float(str(p).replace('$','')), px)
    
get_prices_from_craigslist('rv', 'newyork')

https://newyork.craigslist.org/search/sss?sort=rel&query=rv


[15000.0,
 15000.0,
 18000.0,
 18000.0,
 19999.0,
 19999.0,
 5800.0,
 5800.0,
 750.0,
 750.0,
 450.0,
 450.0,
 400.0,
 400.0,
 60.0,
 60.0,
 25.0,
 25.0,
 49.0,
 49.0,
 135.0,
 135.0,
 310.0,
 310.0,
 50.0,
 50.0,
 310.0,
 310.0,
 20000.0,
 20000.0,
 2680.0,
 2680.0,
 18000.0,
 18000.0,
 9000.0,
 9000.0,
 8900.0,
 8900.0,
 75.0,
 75.0,
 900.0,
 900.0,
 1543.0,
 1543.0,
 18000.0,
 18000.0,
 20.0,
 20.0,
 6800.0,
 6800.0,
 85.0,
 85.0,
 85.0,
 85.0,
 85.0,
 85.0,
 100.0,
 100.0,
 100.0,
 100.0,
 450.0,
 450.0,
 15000.0,
 15000.0,
 150.0,
 150.0,
 8995.0,
 8995.0,
 135000.0,
 135000.0,
 85.0,
 85.0,
 2500.0,
 2500.0,
 26000.0,
 26000.0,
 8.0,
 8.0,
 13899.0,
 13899.0,
 13899.0,
 13899.0,
 2545.0,
 2545.0,
 8.0,
 8.0,
 6900.0,
 6900.0,
 23012.0,
 23012.0,
 425.0,
 425.0,
 425.0,
 425.0,
 340.0,
 340.0,
 425.0,
 425.0,
 425.0,
 425.0,
 1095.0,
 1095.0,
 450.0,
 450.0,
 3648.0,
 3648.0,
 950.0,
 950.0,
 6000.0,
 6000.0,
 1500.0,
 1500.0,
 5000.0,
 5000.0,
 5800.0,
 5800.0,
 150.0,
 150.0,
 3

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [99]:
# I found this site super useful so I scraped it
# https://sites.google.com/site/clsiteinfo/area-id-sort

response = requests.get('https://sites.google.com/site/clsiteinfo/area-id-sort')
html = response.text
URLs = Selector(text=html).xpath('//td/a[@rel="nofollow"]/text()').extract()


In [100]:
URLs

[u'http://sfbay.craigslist.org',
 u'http://seattle.craigslist.org',
 u'http://newyork.craigslist.org',
 u'http://boston.craigslist.org',
 u'http://losangeles.craigslist.org',
 u'http://sandiego.craigslist.org',
 u'http://portland.craigslist.org',
 u'http://washingtondc.craigslist.org',
 u'http://chicago.craigslist.org',
 u'http://sacramento.craigslist.org',
 u'http://denver.craigslist.org',
 u'http://atlanta.craigslist.org',
 u'http://austin.craigslist.org',
 u'http://vancouver.craigslist.ca',
 u'http://philadelphia.craigslist.org',
 u'http://phoenix.craigslist.org',
 u'http://minneapolis.craigslist.org',
 u'http://fortlauderdale.craigslist.org',
 u'http://miami.craigslist.org',
 u'http://westpalmbeach.craigslist.org',
 u'http://dallas.craigslist.org',
 u'http://detroit.craigslist.org',
 u'http://houston.craigslist.org',
 u'http://london.craigslist.co.uk',
 u'http://toronto.craigslist.ca',
 u'http://lasvegas.craigslist.org',
 u'http://cleveland.craigslist.org',
 u'http://honolulu.craig

In [101]:
Selector(text=html).xpath('//tr[@height="17"]/td/text()').extract()

[u'\n',
 u'\n',
 u'\n',
 u'\xa0\xa0\xa0\xa0\xa0\xa0 URL',
 u'Site Name',
 u'State/Province',
 u'*Main Category*',
 u'sfo',
 u'1',
 u'san francisco bay area',
 u'California',
 u'*US*',
 u'sea',
 u'2',
 u'seattle-tacoma',
 u'Washington',
 u'*US*',
 u'nyc',
 u'3',
 u'new york city',
 u'New York',
 u'*US*',
 u'bos',
 u'4',
 u'boston',
 u'Massachusetts',
 u'*US*',
 u'lax',
 u'7',
 u'los angeles',
 u'California',
 u'*US*',
 u'sdo',
 u'8',
 u'san diego',
 u'California',
 u'*US*',
 u'pdx',
 u'9',
 u'portland',
 u'Oregon',
 u'*US*',
 u'wdc',
 u'10',
 u'washington',
 u'District of Columbia',
 u'*US*',
 u'chi',
 u'11',
 u'chicago',
 u'Illinois',
 u'*US*',
 u'sac',
 u'12',
 u'sacramento',
 u'California',
 u'*US*',
 u'den',
 u'13',
 u'denver',
 u'Colorado',
 u'*US*',
 u'atl',
 u'14',
 u'atlanta',
 u'Georgia',
 u'*US*',
 u'aus',
 u'15',
 u'austin',
 u'Texas',
 u'*US*',
 u'van',
 u'16',
 u'vancouver',
 u'British Columbia',
 u'*Canada*',
 u'phi',
 u'17',
 u'philadelphia',
 u'Pennsylvania',
 u'*US*',
 

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Run your scraping process, and save your results to a CSV file.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._

## Optional: Interview Questions

---- 

### SQL Practice

1)
We have a deliveries table with 3000 rows

`SELECT * FROM deliveries; 
-- 3000 rows in set (0.05 sec) `

15 of those orders are from a customer with the customer_id_number=32

`SELECT * FROM deliveries WHERE customer_id_number = 32;
-- 15 rows in set (0.10 sec)`

Yet, when we SELECT the number of orders that are not from customer_id_number = 32, we only get 2960 results:

`SELECT * FROM deliveries WHERE customer_id_number <> 32;
-- 2960 rows in set (0.11 sec)`

**Question: What’s wrong? And why might this be the case? Modify your code to fix this. **

2) Construct the following tables:

`mysql> SELECT * FROM Employee;
+--------+----------+--------+
| emp_id | emp_name | salary |
+--------+----------+--------+
| 1      | James    |   2000 |
| 2      | Jack     |   4000 |
| 3      | Henry    |   6000 |
| 4      | Tom      |   8000 |
+--------+----------+--------+
4 rows IN SET (0.00 sec)`


`mysql> SELECT * FROM Department;
+---------+-----------+
| dept_id | dept_name |
+---------+-----------+
| 101     | Sales     |
| 102     | Marketing |
| 103     | Finance   |
| 104     | Accounting   |
+---------+-----------+
3 rows IN SET (0.00 sec)`


`mysql> SELECT * FROM Register;
+--------+---------+
| emp_id | dept_id |
+--------+---------+
|      1 |     101 |
|      2 |     102 |
|      3 |     103 |
|      4 |     102 |
+--------+---------+
4 rows IN SET (0.00 sec)`

** Questions: ** 
- Which employees belong to which department? Show this using one line of code (hint: more than one join) 
- What is the total marketing salary? 
- Using a join, can you show that there are no employees in accounting? 



3) Given an Employee table which has 3 fields – Id (Primary key), Salary and Manager Id, where manager id is the id of the employee that manages the current employee, find all employees that make more than their manager in terms of salary. Create the table and write the code that finds this


--- 
### Predictive Modeling

- What are some differences you would expect in a regression model that minimizes squared error, versus a model that minimizes absolute error? In which cases would each error  metric be appropriate?

- What error metric would you use to evaluate how good a binary classifier is? What if the classes are imbalanced?  What if there are more than 2 groups?

- What are various ways to predict a binary response variable? Can you compare two of them and tell me when one would be more appropriate? What’s the difference logistic regression and SVMs? 

- What is the difference between the loss functions used by SVMs and Logistic Regression? 

- What is R-squared? What are some other metrics that could be better than R-squared and why?

- You run your regression on different subsets of your data, and find that in each subset, the beta value for a certain variable varies wildly. What could be the issue here?


--- 
### Coding Questions 

- Given a sorted array and a number x, find a pair in array whose sum is closest to x. What is the time complexity of your algorithm? 
    
    `Examples:`
        Input: arr[] = {10, 22, 28, 29, 30, 40}, x = 54
        Output: 22 and 30

        Input: arr[] = {1, 3, 4, 7, 10}, x = 15
        Output: 4 and 10
        
- Check out this video on Linear Time Algorithm for finding the median: https://www.youtube.com/watch?v=_xntajCBLoE. Implement your version of this algorithm in Python. 

- Search in an almost sorted array: Given an array which is sorted, but after sorting some elements are moved to either of the adjacent positions, i.e., arr[i] may be present at arr[i+1] or arr[i-1]. Write an efficient function to search an element in this array. Basically the element arr[i] can only be swapped with either arr[i+1] or arr[i-1]. For example consider the array {2, 3, 10, 4, 40}, 4 is moved to next position and 10 is moved to previous position. [Hint: You can do this O(log n) time complexity]

    `Examples: `
        Input: arr[] =  {10, 3, 40, 20, 50, 80, 70}, key = 40
        Output: 2 
        Output is index of 40 in given array

        Input: arr[] =  {10, 3, 40, 20, 50, 80, 70}, key = 90
        Output: -1
        -1 is returned to indicate element is not present